# Traduzindo dataset com o Gemini

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [ ]:
import yaml

def load_yaml_config(file_path='../translator.prompt.yml'):
    """
    Carrega o arquivo YAML e retorna como um dicionário
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        config = yaml.safe_load(file)
    return config

# Carrega a configuração do YAML
translator_config = load_yaml_config()

print("Chaves disponíveis no config:")
for key in translator_config.keys():
    print(f"- {key}")
    print(f"  Tipo: {type(translator_config.get(key))}")

print(f"\nModelo: {translator_config.get('model')}")
print(f"Temperatura: {translator_config.get('modelParameters', {}).get('temperature')}")
print(f"Número de mensagens: {len(translator_config.get('messages', []))}")
print(f"Role da primeira mensagem: {translator_config.get('messages', [{}])[0].get('role')}")


In [ ]:
df = pd.read_csv(
    '../data/SMSSpamCollectionDataset.csv',
    on_bad_lines='skip',
    encoding='latin1'
)

df = df[['label', 'text']]
df['translated_text'] = None
df.info()
print("\nDistribuição dos labels:")
print(df['label'].value_counts())

In [ ]:
from openai import OpenAI

api_key = os.getenv('GEMINI_API_KEY')
if not api_key:
    raise ValueError("A chave de API GEMINI_API_KEY não está definida no ambiente.")
base_url = os.getenv('GEMINI_BASE_URL')
if not base_url:
    raise ValueError("A URL base GEMINI_BASE_URL não está definida no ambiente.")
llm_model = os.getenv('LLM_MODEL'))

translator = OpenAI(
    api_key=os.getenv('GEMINI_API_KEY'),
    base_url=os.getenv('GEMINI_BASE_URL')
)
print("=== OpenAI client configurado com sucesso. ===")


In [ ]:
def translate_text(row):
    """
    Traduz o texto usando a configuração do YAML
    """

    # print(f"Tipo: {row['label']}")
    if pd.isna(row['translated_text']):
        try:
            translated_response = translator.chat.completions.create(
                model='gemma3:1b',
                temperature=translator_config.get('modelParameters').get('temperature'),
                messages=[
                    {
                        'role': translator_config.get('messages')[0].get('role'),
                        'content': translator_config.get('messages')[0].get('content')
                    },
                    {
                        'role': translator_config.get('messages')[1].get('role'),
                        'content': translator_config.get('messages')[1].get('content').replace('{{text}}', row['text'])
                    }
                ]
            )

            translated_text = translated_response.choices[0].message.content
            # print(f"Texto original: {row['text']}")
            # print(f"Texto traduzido: {translated_text}")
            # print("-" * 50)

            row['translated_text'] = translated_text

        except Exception as e:
            # print(f"Erro ao traduzir o texto: {row['text']}")
            # print(f"ERRO: {e}")
            # print("-" * 50)
            row['translated_text'] = None

    else:
        pass

df = df.progress_apply(translate_text, axis=1)